The Iris cubes used in this notebook are publicly available in the [``SciTools/iris-sample-data``](https://github.com/SciTools/iris-sample-data) repository. This notebook is based on the user story described in [Issue #7](https://github.com/CubeBrowser/cube-explorer/issues/7) with the following [image](http://www.wetterzentrale.de/wz/pics/Recm1201.gif) suggested for inspiration.

In [ ]:
import datetime
import iris
import numpy as np
import holoviews as hv
import geoviews as gv
from cartopy import crs
from cartopy import feature as cf
hv.notebook_extension()

## Setting some notebook-wide options

Let's start by setting some normalization options (discussed below) and always enable colorbars for the elements we will be displaying:

In [ ]:
iris.FUTURE.strict_grib_load = True
%opts Image {+framewise} [colorbar=True] Contours [colorbar=True] {+framewise} Curve [xrotation=60]

Note that it is easy to set global defaults for a project allowing any suitable settings can be made into a default on a per-element basis. Now lets specify the maximum number of frames we will be displaying:

In [ ]:
%output max_frames=1000 


<div class="alert alert-info" role="alert">When working on a live server append ``widgets='live'`` to the line above for greatly improved performance and memory usage </div>


## Loading our first cube

Here is the summary of the first cube containing some surface temperature data:

In [ ]:
iris_cube = iris.load_cube(iris.sample_data_path('GloSea4', 'ensemble_001.pp'))
iris_cube.coord('latitude').guess_bounds()
iris_cube.coord('longitude').guess_bounds()

In [ ]:
print iris_cube.summary()

Now we can wrap this Iris cube in a ``Dataset``:

In [ ]:
surface_temperature = hv.Dataset(iris_cube)
surface_temperature

# A Simple example

Here is a simple example of viewing the ``surface_temperature`` cube over time with a single line of code. In HoloViews, this datastructure is a ``HoloMap`` of ``Image`` elements:

In [ ]:
surface_temperature.to.image(['longitude', 'latitude'])

You can drag the slider to view the surface temperature at different times. Here is how you can view the values of time in the cube via the HoloViews API:

In [ ]:
surface_temperature.dimension_values('time')

The times shown in the slider are long making the text rather small. We can use the fact that all times are recorded in the year 2011 on the 16th of each month to shorten these dates. Defining how all dates should be formatted as follows will help with readability:

In [ ]:
hv.Dimension.type_formatters[datetime.datetime] = "%m/%y %Hh"

Now let us load a cube showing the pre-industrial air temperature:

In [ ]:
air_temperature = hv.Dataset(iris.load_cube(iris.sample_data_path('pre-industrial.pp')),
                             group='Pre-industrial air temperature')
air_temperature.data.coord('longitude').guess_bounds()
air_temperature.data.coord('latitude').guess_bounds()
air_temperature     # Use air_temperature.data.summary() to see the Iris summary (.data is the Iris cube)

Note that we have the ``air_temperature`` available over ``longitude`` and ``latitude`` but *not* the ``time`` dimensions. As a result, this cube is a single frame when visualized as a temperature map.

In [ ]:
(surface_temperature.to.image(['longitude', 'latitude'])+
 air_temperature.to.image(['longitude', 'latitude'])(plot=dict(projection=crs.PlateCarree())))

Next is a fairly involved example that plots data side-by-side in a ``Layout`` without using the ``+`` operator. 

This shows how complex plots can be generated with little code and also demonstrates how different HoloViews elements can be combined together. In the following visualization, the curve is a sample of the ``surface_temperature`` at longitude and latitude *(0,10)*:

In [ ]:
%%opts Layout [fig_inches=(12,7)] Curve [aspect=2 xticks=4 xrotation=20] Points (color=2) Overlay [aspect='equal']
%%opts Image [projection=crs.PlateCarree()]
# Sample the surface_temperature at (0,10)
temp_curve = hv.Curve(surface_temperature.select(latitude=0, longitude=0), kdims=['time'])
# Show surface_temperature and air_temperature with Point (0,10) marked
temp_maps = [cb.to.image(['longitude', 'latitude']) * gv.Points([(0,10)]) 
             for cb in [surface_temperature, air_temperature]]
# Show everything in a two column layout
hv.Layout(temp_maps + [temp_curve]).cols(2).display('all')

## Overlaying data and normalization

Lets view the surface temperatures together with the global coastline:

In [ ]:
%%opts Image [projection=crs.Geostationary()] (cmap='Greens')
surface_temperature.to.image(['longitude', 'latitude']) * gv.Feature(cf.COASTLINE)

Notice that every frame uses the full dynamic range of the Greens color map. This is because normalization is set to ``+framewise`` at the top of the notebook which means every frame is normalized independently. 

To control normalization, we need to decide on the normalization limits. Let's see the maximum temperature in the cube:

In [ ]:
max_surface_temp = surface_temperature.data.data.max()
max_surface_temp

In [ ]:
%%opts Image [projection=crs.Geostationary()] (cmap='Greens')
# Declare a humidity dimension with a range from 0 to 0.01
surface_temp_dim = hv.Dimension('surface_temperature', range=(300, max_surface_temp))
# Use it to declare the value dimension of a Dataset
(hv.Dataset(surface_temperature, vdims=[surface_temp_dim]).to.image(['longitude', 'latitude']) * gv.Feature(cf.COASTLINE))

By specifying the normalization range we can reveal different aspects of the data. In the example above we can see a warming effect over time as the dark green areas close to the bottom of the normalization range (200K) vanish. Values outside this range are clipped to the ends of the color map.

Lastly, here is a demo of a conversion from ``surface_temperature`` to ``Contours``:

In [ ]:
%%opts Contours [levels=10]
(surface_temperature.to.contours(['longitude', 'latitude']) * gv.Feature(cf.COASTLINE))